In [3]:
# enable auto‑reload so edits in src/ propagate immediately
%load_ext autoreload
%autoreload 2

# now import everything in one line:
from openiex import (
    SystemConfig, Ion, Protein, ExchangeSystem,
    Method, validate_method,
    initialize_profiles, SimulationTracker,
    method_duration, run_simulation,
    save_simulation, load_simulation, plot_profiles
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
ions = {
    "Cl": Ion("Cl", D=1e-10, Kd=1.0),
    "Ac": Ion("Ac", D=1e-10, Kd=1.0)
}

proteins = {
    "em": Protein("em", D=5e-12, Kd=1.0, sigma=100, nu=5),
    "fu": Protein("fu", D=5e-12, Kd=1.0, sigma=100, nu=5)
}

config = SystemConfig(
    bed_height=0.2,
    column_radius=0.004,
    Lambda=0.5,
    epsilon_i=0.5,
    epsilon_p=0.5,
    Nz=30
)

system = ExchangeSystem(ions, proteins, config)
species_list = list(system.species.keys())
system.set_equilibrium("Cl", "Ac", K_eq_val=2.0, k_ads_val=1.0)
system.set_equilibrium("em", "Cl", K_eq_val=5e3, k_ads_val=1e4)
system.set_equilibrium("em", "Ac", K_eq_val=5e-2, k_ads_val=1e3)
system.set_equilibrium("fu", "Cl", K_eq_val=1e4, k_ads_val=1e4)
system.set_equilibrium("fu", "Ac", K_eq_val=1e-1, k_ads_val=1e3)
system.check_equilibria()

All required equilibria are defined.


In [7]:
buffers = {
    "Load": {"Cl": 0.04, "Ac": 0.0, "em": 0, "fu": 5e16},
    "A": {"Cl": 0.04, "Ac": 0.0, "em": 0.0, "fu": 0.0},
    "B": {"Cl": 0.04, "Ac": 0.3, "em": 0.0, "fu": 0.0},
    "Spike": {"Cl": 1.0, "Ac": 0.0, "em": 0.0, "fu": 0.0},
}

# CV = total column volume with no interstitial void correction
blocks = [
    {"buffer_A": "A", "buffer_B": "B", "start_B": 0.0, "end_B": 0.0, "duration_CV": 1, "flow_rate_mL_min": 2.7},
    {"buffer_A": "Spike", "buffer_B": "B", "start_B": 0.0, "end_B": 0.0, "duration_CV": 0.02, "flow_rate_mL_min": 2.7},
    {"buffer_A": "A", "buffer_B": "B", "start_B": 0.0, "end_B": 0.0, "duration_CV": 2, "flow_rate_mL_min": 2.7},
]

method = Method(buffers, blocks)

validate_method(method, system)

In [9]:
import numpy as np

initial_conditions = {
    "Cl": {"C": 0.04, "Q": system.config.Lambda},
}
y0 = initialize_profiles(initial_conditions, system)
t_final = method_duration(method, system)
t_eval = np.linspace(0, t_final, 100)

sol = run_simulation(
    method, system, y0, t_eval,
    integrator='BDF',   # <-- no collision with `chromat_method`
    rtol=1e-3,
    atol=1e-5,
    max_step=0.1
)

Simulating:   0%|          | 0/674.5395677373806 [00:00<?, ?s/s]

RuntimeError: Time 674.5395677373806 exceeded defined method blocks — simulation method incomplete?